# **Customer Churn Prediction using ANN**
This notebook demonstrates the implementation of an Artificial Neural Network (ANN) to predict customer churn based on various features such as credit score, geography, age, and account activity. 

---

## **1. Importing Required Libraries**
We begin by importing the necessary Python libraries for data handling, preprocessing, machine learning, and deep learning.


In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

2025-03-14 19:42:31.255966: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 19:42:31.261208: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 19:42:31.319666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741981351.436286   20061 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741981351.473670   20061 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1741981351.553064   20061 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

---

## 2. Loading and Using the Model for Prediction

Loading the Model and Preprocessing Tools.We reload the trained ANN model, encoders, and scaler.


In [2]:
### Load the trained model, scaler pickle,onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo_new.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('label_encoder_gender_new.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

2025-03-14 19:43:10.247104: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


---

## 3. Preparing Input Data for Prediction

We create an example customer profile.We load the dataset, which contains customer information, and display the first few rows.

In [41]:
# Example input data
input_data = {
    'CreditScore': 300,
    'Geography': 'Germany',
    'Gender': 'Male',
    'Age': 27,
    'Tenure': 3,
    'Balance': 200,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 5000
}

One-hot encoding is applied to the Geography field.

In [42]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Geography_France  Geography_Germany  Geography_Spain
0               0.0                1.0              0.0

We convert the input dictionary to a DataFrame and encode categorical features.


In [43]:
input_df=pd.DataFrame([input_data])
input_df

CreditScore Geography Gender  Age  Tenure  Balance  NumOfProducts  \
0          300   Germany   Male   27       3      200              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1             5000

In [44]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

CreditScore Geography  Gender  Age  Tenure  Balance  NumOfProducts  \
0          300   Germany       1   27       3      200              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1             5000

In [45]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

CreditScore  Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          300       1   27       3      200              2          1   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0               1             5000               0.0                1.0   

   Geography_Spain  
0              0.0

The input data is then scaled.

In [46]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-3.64930447,  0.91324755, -1.13114808, -0.69539349, -1.21526836,
         0.80843615,  0.64920267,  0.97481699, -1.65923237, -0.99850112,
         1.72572313, -0.57638802]])

---

### 4.Making Predictions
We pass the processed input data to the trained model.

In [47]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


array([[0.02788707]], dtype=float32)

In [48]:
prediction_proba = prediction[0][0]

In [49]:
prediction_proba

np.float32(0.027887067)

The prediction probability is interpreted as follows:

In [50]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.


### Conclusion

In this notebook, we successfully implemented an ANN model to predict customer churn. The workflow involved:

> Data preprocessing (handling categorical variables, scaling numerical values).

> Training an ANN model using TensorFlow/Keras.

> Saving and loading preprocessing tools.

> Making predictions on new customer data.

This model can be extended further by optimizing hyperparameters, adding more layers, or incorporating additional features. 🚀